# Experiment 1-C

Improvements to the text matcher. Stemming, etc. 

In [1]:
import json
from collections import Counter
import pandas as pd
from nltk.corpus import names
import nltk
import re 
import os
import difflib 
import logging
import itertools
from nltk.util import ngrams 
from nltk.stem import LancasterStemmer
from difflib import SequenceMatcher
from string import punctuation
from termcolor import colored
from IPython.display import clear_output
%matplotlib inline

In [4]:
# Load the data. 
with open('txt/middlemarch.json') as f: 
    rawCriticism = f.readlines()

# Parse the data. 
data = [json.loads(line) for line in rawCriticism]

# Load Middlemarch
with open('middlemarch.txt') as f: 
    rawMM = f.read()

In [11]:
class Text: 
    def __init__(self, raw_text, label, removeStopwords=True): 
        if type(raw_text) == list: 
            # JSTOR critical works come in lists, where each item represents a page. 
            self.text = ' \n '.join(raw_text)
        else: 
            self.text = raw_text
        self.label = label
        self.tokens = self.getTokens(removeStopwords)
        self.trigrams = self.ngrams(3)
        
    def getTokens(self, removeStopwords=True): 
        """ Tokenizes the text, breaking it up into words, removing punctuation. """
        tokenizer = nltk.RegexpTokenizer('[a-zA-Z]\w+\'?\w*') # A custom regex tokenizer. 
        #tokenizer = nltk.RegexpTokenizer('\w+|\$[\d\.]+|\S+') # A custom regex tokenizer. 
        spans = list(tokenizer.span_tokenize(self.text))
        # Take note of how many spans there are in the text
        #print(spans)
        self.length = spans[-1][-1] 
        tokens = tokenizer.tokenize(self.text)
        tokens = [ token.lower() for token in tokens ] # make them lowercase
        stemmer = LancasterStemmer()
        tokens = [ stemmer.stem(token) for token in tokens ]
        if not removeStopwords: 
            self.spans = spans
            return tokens
        tokenSpans = list(zip(tokens, spans)) # zip it up
        stopwords = nltk.corpus.stopwords.words('english') # get stopwords
        tokenSpans = [ token for token in tokenSpans if token[0] not in stopwords ] # remove stopwords from zip
        self.spans = [ x[1] for x in tokenSpans ] # unzip; get spans
        return [ x[0] for x in tokenSpans ] # unzip; get tokens
    
    def ngrams(self, n): 
        """ Returns ngrams for the text."""
        return list(ngrams(self.tokens, n))

class Matcher: 
    def __init__(self, textObjA, textObjB, threshold=5, ngramSize=3, removeStopwords=True):
        """
        Takes as input two Text() objects, and matches between them.
        """
        self.threshold = threshold
        self.ngramSize = ngramSize
        
        #self.textA, self.textB = Text(fileA, removeStopwords=removeStopwords), \
        #        Text(fileB, removeStopwords=removeStopwords)
        self.textA = textObjA
        self.textB = textObjB 
        
        self.textAgrams = self.textA.ngrams(ngramSize)
        self.textBgrams = self.textB.ngrams(ngramSize)

        self.locationsA = []
        self.locationsB = []

    def getContext(self, text, start, length, context): 
        match = self.getTokensText(text, start, length)
        before = self.getTokensText(text, start-context, context)
        after = self.getTokensText(text, start+length, context)
        match = colored(match, 'red')
        out = " ".join([before, match, after])
        out = out.replace('\n', ' ') # Replace newlines with spaces. 
        out = re.sub('\s+', ' ', out)
        return out

    def getTokensText(self, text, start, length):  
        """ Looks up the passage in the original text, using its spans. """
        matchTokens = text.tokens[start:start+length]
        spans = text.spans[start:start+length]
        if len(spans) == 0: 
            # Don't try to get text or context beyond the end of a text. 
            passage = ""
        else: 
            passage = text.text[spans[0][0]:spans[-1][-1]]
        return passage 

    def getLocations(self, text, start, length, asPercentages=False): 
        """ Gets the numeric locations of the match. """
        spans = text.spans[start:start+length]
        if asPercentages: 
            locations = (spans[0][0]/text.length, spans[-1][-1]/text.length)
        else: 
            locations = (spans[0][0], spans[-1][-1])
        return locations

    def getMatch(self, match, textA, textB, context): 
        length = match.size + self.ngramSize - 1 # offset according to nGram size 
        wordsA = self.getContext(textA, match.a, length, context)
        wordsB = self.getContext(textB, match.b, length, context)
        spansA = self.getLocations(textA, match.a, length)
        spansB = self.getLocations(textB, match.b, length)
        self.locationsA.append(spansA)
        self.locationsB.append(spansB)
        line1 = ('%s: %s %s' % (colored(textA.label, 'green'), spansA, wordsA) )
        line2 = ('%s: %s %s' % (colored(textB.label, 'green'), spansB, wordsB) )
        return line1 + '\n' + line2

    def match(self): 
        """
        This does the main work of finding matching n-gram sequences between
        the texts.
        """
        sequence = SequenceMatcher(None,self.textAgrams,self.textBgrams)
        matchingBlocks = sequence.get_matching_blocks()

        # Only return the matching sequences that are higher than the 
        # threshold given by the user. 
        highMatchingBlocks = [match for match in matchingBlocks if match.size > self.threshold]
    
        numBlocks = len(highMatchingBlocks)
        self.numMatches = numBlocks
        
        if numBlocks > 0: 
            print('%s total matches found.' % numBlocks, flush=True)

        for num, match in enumerate(highMatchingBlocks): 
            print('match: ', match)
            out = self.getMatch(match, self.textA, self.textB, 5)
            print('\n')
            print('match %s:' % (num+1), flush=True)
            print(out, flush=True)

        return self.numMatches, self.locationsA, self.locationsB

In [5]:
mm = Text(rawMM, 'Middlemarch')

In [8]:
test1 = Text(data[0]['ocr'], 'test1')

In [13]:
stemmer = LancasterStemmer()
stemmer.stem('labourer')

'labo'

In [12]:
Matcher(mm, test1).match()

24 total matches found.
match:  Match(a=550, b=971, size=37)


match 1:
Middlemarch: (5809, 6218) interest in gimp and artificial protrusions of drapery mind was theoretic, and yearned by its nature after some lofty conception of the world which might frankly include the parish of Tipton and her own rule of conduct there; she was enamoured of intensity and greatness, and rash in embracing whatever seemed to her to have those aspects; likely to seek martyrdom, to make retractations, and then to incur martyrdom after all in a quarter where she had not sought Certainly such elements in the character of a marriageable girl
test1: (10484, 10893) closely, as the first description of Dorothea shows mind was theoretic, and yearned by its nature after some lofty conception of the world which might frankly include the parish of Tipton and her own rule of conduct there; she was enamoured of intensity and greatness, and rash in embracing whatever seemed to her to have those aspects; likely to seek

(24,
 [(5809, 6218),
  (8751, 8851),
  (8890, 9046),
  (57013, 57100),
  (83868, 83999),
  (116900, 117085),
  (117160, 117594),
  (192301, 192441),
  (195148, 195588),
  (402604, 402726),
  (411725, 412177),
  (449403, 449911),
  (449924, 450049),
  (450145, 450244),
  (1575265, 1575374),
  (1576340, 1576437),
  (1648982, 1649408),
  (1649495, 1649704),
  (1688955, 1689089),
  (1689907, 1690146),
  (1690158, 1690307),
  (1708999, 1709204),
  (1709221, 1709342),
  (1793142, 1793447)],
 [(10484, 10893),
  (11022, 11122),
  (11610, 11765),
  (12440, 12527),
  (13752, 13883),
  (29395, 29580),
  (29659, 30092),
  (34726, 34866),
  (35477, 35916),
  (37725, 37845),
  (42065, 42515),
  (42700, 43207),
  (43222, 43347),
  (43367, 43466),
  (44773, 44882),
  (44892, 44989),
  (45826, 46249),
  (46338, 46542),
  (47322, 47456),
  (47461, 47699),
  (47713, 47862),
  (49114, 49317),
  (49335, 49454),
  (50226, 50531)])